In [1]:
import pandas as pd
#import modin.pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import math
pd.set_option('display.max_columns', None)

In [2]:
df = pd.read_csv('2021_public_lar.csv',usecols=['state_code','derived_loan_product_type','derived_dwelling_category'
                                                ,'derived_race','applicant_race_1','derived_sex','action_taken',
                                                'loan_purpose','business_or_commercial_purpose',
                                                'loan_amount','combined_loan_to_value_ratio','interest_rate',
                                                'loan_term','property_value','occupancy_type','income',
                                                'debt_to_income_ratio','applicant_credit_score_type','denial_reason_1'])

C:\Users\josep\AppData\Local\Temp\ipykernel_14552\361376006.py:1: DtypeWarning: Columns (22,23,31,38,46) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('2021_public_lar.csv',usecols=['state_code','derived_loan_product_type','derived_dwelling_category'


In [21]:
# Look only at NY
# conventional loan:  first lien
# home purchase
# not commercial/business
# single family home
# principle residence/Owner occupied 

df_ny = df[(df["state_code"] == 'NY')]
df_ny = df_ny[df_ny['derived_loan_product_type'] == 'Conventional:First Lien']
df_ny = df_ny.loc[df_ny['loan_purpose'].isin([1])]
df_ny = df_ny.loc[df_ny['business_or_commercial_purpose'].isin([2])]
df_ny = df_ny[df_ny['derived_dwelling_category'] == 'Single Family (1-4 Units):Site-Built']
df_ny = df_ny.loc[df_ny['occupancy_type'].isin([1])]
df_ny['combined_loan_to_value_ratio'] = df_ny['combined_loan_to_value_ratio'].astype(str).astype(float)
df_ny['interest_rate'] = df_ny['interest_rate'].astype(str).astype(float)
df_ny = df_ny.loc[df_ny['action_taken'].isin([1,3])]
df_ny['action_taken'] = df['action_taken'].replace({3: 0})


In [20]:
df_ny.dtypes

state_code                         object
derived_loan_product_type          object
derived_dwelling_category          object
derived_race                       object
derived_sex                        object
action_taken                        int64
loan_purpose                        int64
business_or_commercial_purpose      int64
loan_amount                         int64
combined_loan_to_value_ratio      float64
interest_rate                     float64
loan_term                          object
property_value                     object
occupancy_type                      int64
income                            float64
debt_to_income_ratio               object
applicant_credit_score_type         int64
applicant_race_1                  float64
denial_reason_1                     int64
dtype: object

In [24]:
df_ny[df_ny['action_taken'] == 1]

,state_code,derived_loan_product_type,derived_dwelling_category,derived_race,derived_sex,action_taken,loan_purpose,business_or_commercial_purpose,loan_amount,combined_loan_to_value_ratio,interest_rate,loan_term,property_value,occupancy_type,income,debt_to_income_ratio,applicant_credit_score_type,applicant_race_1,denial_reason_1
1,NY,Conventional:First Lien,Single Family (1-4 Units):Site-Built,White,Male,1,1,2,95000,95.000,2.875,360,105000,1,35.0,39,2,5.0,10
2,NY,Conventional:First Lien,Single Family (1-4 Units):Site-Built,White,Male,1,1,2,95000,96.999,3.125,360,95000,1,24.0,38,3,5.0,10
4,NY,Conventional:First Lien,Single Family (1-4 Units):Site-Built,White,Male,1,1,2,105000,90.000,3.125,360,115000,1,80.0,<20%,1,5.0,10
9,NY,Conventional:First Lien,Single Family (1-4 Units):Site-Built,White,Joint,1,1,2,255000,80.000,3.000,360,315000,1,68.0,36,1,5.0,10
10,NY,Conventional:First Lien,Single Family (1-4 Units):Site-Built,White,Male,1,1,2,305000,89.820,3.000,360,335000,1,107.0,38,1,5.0,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26124546,NY,Conventional:First Lien,Single Family (1-4 Units):Site-Built,White,Joint,1,1,2,135000,80.000,3.250,360,165000.0,1,108.0,20%-<30%,2,5.0,10
26124547,NY,Conventional:First Lien,Single Family (1-4 Units):Site-Built,White,Female,1,1,2,165000,91.954,3.125,360,175000.0,1,44.0,30%-<36%,3,5.0,10
26124548,NY,Conventional:First Lien,Single Family (1-4 Units):Site-Built,Race Not Available,Male,1,1,2,165000,97.000,2.875,360,165000.0,1,45.0,40,2,6.0,10
26124549,NY,Conventional:First Lien,Single Family (1-4 Units):Site-Built,White,Male,1,1,2,255000,99.953,3.125,360,265000.0,1,64.0,30%-<36%,1,5.0,10


In [38]:
df_ny_approved = df_ny[df_ny['action_taken'].isin([1])]

In [39]:
data = pd.get_dummies(df_ny, columns=['derived_race'], drop_first=True)
data = data.rename(columns={'derived_race_American Indian or Alaska Native': 'AI_AN'})
data = data.rename(columns={'derived_race_Black or African American': 'Black'})
data = data.rename(columns={'derived_race_Native Hawaiian or Other Pacific Islander': 'NH_PI'})
data.columns

Index(['state_code', 'derived_loan_product_type', 'derived_dwelling_category',
       'derived_sex', 'action_taken', 'loan_purpose',
       'business_or_commercial_purpose', 'loan_amount',
       'combined_loan_to_value_ratio', 'interest_rate', 'loan_term',
       'property_value', 'occupancy_type', 'income', 'debt_to_income_ratio',
       'applicant_credit_score_type', 'applicant_race_1', 'denial_reason_1',
       'AI_AN', 'derived_race_Asian', 'Black',
       'derived_race_Free Form Text Only', 'derived_race_Joint', 'NH_PI',
       'derived_race_Race Not Available', 'derived_race_White'],
      dtype='object')

In [40]:
data

,state_code,derived_loan_product_type,derived_dwelling_category,derived_sex,action_taken,loan_purpose,business_or_commercial_purpose,loan_amount,combined_loan_to_value_ratio,interest_rate,loan_term,property_value,occupancy_type,income,debt_to_income_ratio,applicant_credit_score_type,applicant_race_1,denial_reason_1,AI_AN,derived_race_Asian,Black,derived_race_Free Form Text Only,derived_race_Joint,NH_PI,derived_race_Race Not Available,derived_race_White
1,NY,Conventional:First Lien,Single Family (1-4 Units):Site-Built,Male,1,1,2,95000,95.000,2.875,360,105000,1,35.0,39,2,5.0,10,0,0,0,0,0,0,0,1
2,NY,Conventional:First Lien,Single Family (1-4 Units):Site-Built,Male,1,1,2,95000,96.999,3.125,360,95000,1,24.0,38,3,5.0,10,0,0,0,0,0,0,0,1
4,NY,Conventional:First Lien,Single Family (1-4 Units):Site-Built,Male,1,1,2,105000,90.000,3.125,360,115000,1,80.0,<20%,1,5.0,10,0,0,0,0,0,0,0,1
9,NY,Conventional:First Lien,Single Family (1-4 Units):Site-Built,Joint,1,1,2,255000,80.000,3.000,360,315000,1,68.0,36,1,5.0,10,0,0,0,0,0,0,0,1
10,NY,Conventional:First Lien,Single Family (1-4 Units):Site-Built,Male,1,1,2,305000,89.820,3.000,360,335000,1,107.0,38,1,5.0,10,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26124546,NY,Conventional:First Lien,Single Family (1-4 Units):Site-Built,Joint,1,1,2,135000,80.000,3.250,360,165000.0,1,108.0,20%-<30%,2,5.0,10,0,0,0,0,0,0,0,1
26124547,NY,Conventional:First Lien,Single Family (1-4 Units):Site-Built,Female,1,1,2,165000,91.954,3.125,360,175000.0,1,44.0,30%-<36%,3,5.0,10,0,0,0,0,0,0,0,1
26124548,NY,Conventional:First Lien,Single Family (1-4 Units):Site-Built,Male,1,1,2,165000,97.000,2.875,360,165000.0,1,45.0,40,2,6.0,10,0,0,0,0,0,0,1,0
26124549,NY,Conventional:First Lien,Single Family (1-4 Units):Site-Built,Male,1,1,2,255000,99.953,3.125,360,265000.0,1,64.0,30%-<36%,1,5.0,10,0,0,0,0,0,0,0,1


In [51]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

# adjusting loan_approval to have binary outcome, that is action taken (1 is approved, 0 is rejected), 'race', 'income', 'credit_score', and 'loan_amount'
# logistic regression model with just race

# interest rate when added cause perfect separation error
# model is unable to converge due to complete or quasi-complete separation in the data. 
# This occurs when a combination of predictor variables perfectly separates the outcome variable into two groups, 
# such that there are no overlapping observations between the groups.
formula = 'action_taken ~ AI_AN + derived_race_Asian + Black + NH_PI + derived_race_White'
logit_model1 = smf.logit(formula, data=data).fit()

# Display the results
print(type(logit_model1))
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
print(logit_model1.summary())

Optimization terminated successfully.
         Current function value: 0.225162
         Iterations 10
<class 'statsmodels.discrete.discrete_model.BinaryResultsWrapper'>
                           Logit Regression Results                           
Dep. Variable:           action_taken   No. Observations:               135829
Model:                          Logit   Df Residuals:                   135802
Method:                           MLE   Df Model:                           26
Date:                Sat, 25 Mar 2023   Pseudo R-squ.:                  0.2014
Time:                        17:07:41   Log-Likelihood:                -30583.
converged:                       True   LL-Null:                       -38297.
Covariance Type:            nonrobust   LLR p-value:                     0.000
                                       coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------------

In [ ]:
formula2 = 'action_taken ~ AI_AN + derived_race_Asian + Black + NH_PI + derived_race_White + income + combined_loan_to_value_ratio + loan_amount + debt_to_income_ratio'
# Fit the logistic regression model
logit_model2 = smf.logit(formula2, data=data).fit()

# Display the results
print(type(logit_model2))
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
print(logit_model2.summary())
